# 1. Preparing Data

In [3]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, ks_2samp

## Contract data

In [4]:
df = pd.read_csv('../data/raw/voc_person_contracts.csv.gz')
df.head()

,vocop_id,full_name,first_name,patronymic,family_name_prefix,family_name,place_of_origin,place_id,disambiguated_person,person_cluster_id,...,changed_ship_at_cape_voyage_id,return_voyage_id,remark,source_id,scan_permalink,full_name_normalized,first_name_normalized,patronymic_normalized,family_name_prefix_normalized,family_name_normalized
0,104944,Adriaen van Renteregem,Adriaen,NaN,van,Renteregem,Wassenaar,141865.0,1.0,415682.0,...,NaN,96993,NaN,3339,NL-HaNA_1.04.02_12674_0192.jpg,Adriaan van Renteregem,Adriaan,NaN,van,Renteregem
1,104945,Adriaen van der Meulen,Adriaen,NaN,van der,Meulen,Cooltiensplate,25576.0,1.0,406243.0,...,NaN,96942,NaN,3339,NL-HaNA_1.04.02_12674_0194.jpg,Adriaan van der Meul,Adriaan,NaN,van der,Meul
2,104946,Arnoldus Coutrel,Arnoldus,NaN,NaN,Coutrel,Antwerpen,6486.0,1.0,1.0,...,NaN,97025,NaN,3339,NL-HaNA_1.04.02_12674_0196.jpg,Arnoldus Koetrel,Arnoldus,NaN,NaN,Koetrel
3,104947,Albert Coolman,Albert,NaN,NaN,Coolman,Amsterdam,5532.0,1.0,77235.0,...,NaN,-1,NaN,3339,NL-HaNA_1.04.02_12674_0198.jpg,Albert Koolman,Albert,NaN,NaN,Koolman
4,104948,Anthonij Bonel,Anthonij,NaN,NaN,Bonel,Amsterdam,5532.0,1.0,65732.0,...,NaN,-1,NaN,3339,NL-HaNA_1.04.02_12674_0200.jpg,Antoni Bonel,Antoni,NaN,NaN,Bonel


In [5]:
print(f'Total number of contracts: {df.shape[0]:,}')

Total number of contracts: 774,200


### Variables

- __vocop_id__: the unique id for each contract <br>
- __disambiguated_person__: indicates whether the person is disambiguated (0 = no, 1 = yes). <br>
- __person_cluster_id__: id that refers to unique person. If a person is disambiguated, multiple contracts can belong to one person <br>
- __date_begin_contract__: start date of contract <br>
- __outward_voyage_id__: unique id of outward voyage.

In [6]:
# select relevant columns

df = df[['vocop_id', 'disambiguated_person', 'person_cluster_id', 'date_begin_contract', 'outward_voyage_id']]

In [7]:
# Remove contracts without a start data
nans = df['date_begin_contract'].isnull().sum()
print(f'Number of contacts with missing data in start date field: {nans}')
df = df.dropna(subset=['date_begin_contract'])

# Only select disambiguated records
records = df.loc[df['disambiguated_person'] == 1, 'person_cluster_id'].astype(int).values
print(f'Number of disambiguated records: {len(records):,}')

# Construct year variable
# Pandas does not like old dates, so rather than using Datetime, we extract it from the string
df['year'] = df['date_begin_contract'].str.split('-', expand=True)[0].astype(int)
df = df.sort_values(by='year')
assert df["year"].min() == 1633 and df["year"].max() == 1794

df.to_csv('../data/voyages_clean.csv', index=False)

Number of contacts with missing data in start date field: 257
Number of disambiguated records: 546,973


In [8]:
sailors_df = df[df['outward_voyage_id'] != 0]

In [9]:
contract_counts = sailors_df.groupby('outward_voyage_id')['vocop_id'].count()

# Dutch-Asiatic Shipping (DAS)

In [10]:
das = pd.read_excel('../data/raw/das.xlsx')

das = das[das['voyRGPDeel'] == 166] #only outbound voyages
das = das[(das['voydepartureY'] >= 1633) & (das['voydepartureY'] <= 1794)] # same time period as contracts

outward_voyage_ids = das['voyId'].values
print(f'number of das records: {das.shape[0]}')

print(f'number of outward voyages: {len(outward_voyage_ids)}')

number of das records: 4352
number of outward voyages: 4352


In [11]:
das_onboard = pd.read_excel('../data/raw/das.xlsx', sheet_name='onboard')
das_onboard = das_onboard[das_onboard['onbVoyageId'].isin(outward_voyage_ids)] # select outbound vogages in time period

- `onbI`: The number on board at departure. 
- `onbII`: The number dying between the Netherlands and the Cape. 
- `onbIII`: The number who left the ship at the Cape.  
- `onbIV`: The number who came on board at the Cape.
- `onbV`: The number dying on the whole voyage. 
- `onbVI`: The number on board on arrival in Asia.

For our purposes, we only need `onbI` and `onbIV`. These columns contain dirty data, so we need to extract only the integers.

In [12]:
print(das_onboard['onbVoyageId'].nunique())

# not all onboard records have voyage ids that match the information on year of departure and such. 4787 vs 4505
# therefore we need to use the means for the travels in onboard and multiple with voyages in das to estimate total number of passengers.

4223


In [13]:
das_onboard['onbI'].value_counts().tail(20)

322                    1
515                    1
408                    1
393                    1
± 249                  1
25200                  1
775                    1
376                    1
2018-04-03 00:00:00    1
430                    1
250?                   1
770                    1
327                    1
313(?)                 1
383                    1
251                    1
367                    1
346                    1
339                    1
375                    1
Name: onbI, dtype: int64

- Take first value when there are two values (separated with /)
- remove ? and ± using regex
- remove date 2018-04-03 00:00:00
- remove 25200 (this is probably 252) 

In [14]:
das_onboard['onbIV'].value_counts().tail(20)

144    1
113    1
165    1
62     1
96     1
70     1
172    1
91     1
54     1
71     1
2*     1
82     1
26*    1
49     1
107    1
54*    1
0*     1
10*    1
3*     1
100    1
Name: onbIV, dtype: int64

- replace * with 0

In [18]:
das_onboard = das_onboard.loc[das_onboard['onbI'].apply(lambda x: isinstance(x, (int, str)))]
das_onboard['onbIV'] =  das_onboard['onbIV'].astype(str).str.replace('*', '0', regex=False)
das_onboard['onbI'] = das_onboard['onbI'].astype(str).str.extract('(\d+)') #extract digits (400 / 500, +- 400)
das_onboard['onbIV'] = das_onboard['onbIV'].astype(str).str.extract('(\d+)')
das_onboard['onbIV'] = das_onboard['onbIV'].fillna(0) #fill missing with 0 so that we can more easily add them together

In [19]:
das_onboard['onbCategory'].value_counts()

Seafarers     2891
Soldiers      2614
Passengers    1696
Craftsmen     1308
Total         1051
Name: onbCategory, dtype: int64

In [20]:
# add year of departure
das_onboard = pd.merge(
    das_onboard, das[['voyId', 'voydepartureY']], 
    left_on='onbVoyageId', right_on='voyId'
).drop('voyId', axis=1)

In [21]:
# add onboarding in republic with those in cape
das_onboard['counts'] = das_onboard['onbI'].astype(int).add(das_onboard['onbIV'].astype(int))

In [22]:
bins = list(range(1633, 1795, 25)) + [1795]
date_ranges = [(bins[i-1], bins[i]) for i in range(1, len(bins))]
print(f'These are the date ranges: {date_ranges}')

These are the date ranges: [(1633, 1658), (1658, 1683), (1683, 1708), (1708, 1733), (1733, 1758), (1758, 1783), (1783, 1795)]


In [24]:
count_table = list()
total_count = 0

das_onboard = das_onboard[das_onboard['onbVoyageId'] != 93096] # remove extreme value
das_onboard = das_onboard[das_onboard['counts'] >= 0] # remove voyages with no passengers, we don't want ghost ships

for index, (start, end) in enumerate(date_ranges):
    d = {}
    das_slice = das[das['voydepartureY'].between(start, end, inclusive="left")]
    df_slice = das_onboard[das_onboard['voydepartureY'].between(start, end, inclusive="left")]
    ships_das = das_slice['voyId'].nunique()
    ships_onboard = df_slice['onbVoyageId'].nunique()
    voyagers = df_slice[df_slice['onbCategory'] != 'Passengers']['counts'].sum() # exclude passengers

    d["period"] = f"{start} - {end}"
    d['ships'] = ships_das
    d['ships_onboard'] = ships_onboard
    d['voyagers_onboard'] = voyagers

    average_pass = voyagers / ships_onboard
    
    totals_pass = average_pass * ships_das
    total_count += totals_pass
    total_count = int(np.round(total_count))
    d['average_voyagers'] = np.round(average_pass, 1)
    d['total_voyagers'] = int(np.round(totals_pass))
    count_table.append(d)
total_count = int(np.round(total_count))

In [25]:
total_counts_df = pd.DataFrame(count_table)

In [27]:
print(total_count)
total_counts_df['total_voyagers'].sum()

952147


952147

In [28]:
total_counts_df.to_csv('../data/total_counts.csv', index=False)